In [18]:
from config_llm import llm_gpt4_mini, embeddings, llm_gpt4o, client_gpt4o, client_gpt4_mini
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings
from llama_index.core.postprocessor import LongContextReorder
reorder = LongContextReorder()

In [19]:
Settings.llm = llm_gpt4o
Settings.embed_model = embeddings
db2 = chromadb.PersistentClient(path="./RagwithMedata")
chroma_collection = db2.get_or_create_collection("RagwithMedata")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
reorder = LongContextReorder()
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embeddings)
my_retriever = index.as_retriever(node_postprocessors=[reorder], similarity_top_k=5)

In [13]:
def RagQuery(query):
    retrieved_sources = my_retriever.retrieve(query)
    response = client_gpt4o.chat.completions.create(
        model="gpt-4-latest",
        messages=[
            {
                "role": "user",
                "content": f"""
            1. Read the provided sources carefully, including all details.
            2. Answer the user’s query using only the provided sources. Me Do not rely on prior knowledge.
            3. If the text does not contain a relevant answer, respond with: "Sorry, the provided text is not relevant to the query."
            4. Cite the source with clickable link using the provided URL.
            5. Make sure to add page number so that the user can verify it.
            User Query: {query}
            Sources:
            {retrieved_sources}
                """
            }
        ],
        temperature=0,
        max_tokens=4000
    )
    return response.choices[0].message.content


In [16]:
from IPython.display import Markdown, display
FirstQuery = RagQuery('What is multi-dimensional poverty?')
display(Markdown(f"<b>{FirstQuery}</b>"))

<b>Multidimensional poverty refers to the condition where individuals or households experience multiple deprivations simultaneously, across various indicators of health, education, and standard of living. The global Multidimensional Poverty Index (MPI) measures acute multidimensional poverty across more than 100 developing countries. It was first launched in 2010 by the Oxford Poverty and Human Development Initiative (OPHI) and the Human Development Report Office of the United Nations Development Programme. The MPI advances Sustainable Development Goal 1, which aims to end poverty in all its forms everywhere, by measuring interconnected deprivations across indicators related to several SDGs. The computation of the global MPI begins by constructing a deprivation profile for each household and person, covering 10 indicators of health, education, and standard of living. For example, a household is considered deprived if any child is stunted or underweight, if any child died in the past five years, or if any school-aged child is not attending school [source](https://hdr.undp.org/content/2024-global-multidimensional-poverty-index-mpi) (Page 6).</b>

In [15]:
SecondQuery = RagQuery('Which country had the most siginficant progress in elevating poverty?')
display(Markdown(f"<b>{SecondQuery}</b>"))

<b>The country that had the most significant progress in elevating poverty, according to the 2024 Global Multidimensional Poverty Index (MPI), is Benin. Between 2017/2018 and 2021/2022, Benin experienced the fastest reduction in MPI values and in the incidence of poverty, decreasing from 66.0 percent to 55.9 percent. This change is significant at the 95 percent confidence level. [Source](https://hdr.undp.org/content/2024-global-multidimensional-poverty-index-mpi) (Page 15).</b>